In [0]:
from sklearn.datasets import make_classification as mc
import pandas as pd


sample = .20 #.20
#training
data,y = mc(n_samples=15_000_00,n_features= 10,n_informative=7) #10_000_000 weights=(.5,.5)
data = pd.DataFrame(data)
y = pd.Series(y,name='stabf')
data = pd.concat((data,y),axis=1)
df_train_spark = spark.createDataFrame(data)

#validation
# data_val,y_val = mc(n_samples=10000,n_features= 5,n_informative=3,weights=(.7,.30)) #10_000_000
# data_val = pd.DataFrame(data_val)
# y_val = pd.Series(y,name='stabf')
# data_val = pd.concat((data_val,y_val),axis=1)
# df_val_spark = spark.createDataFrame(data_val)
df_val_spark = df_train_spark.stat.sampleBy("stabf",fractions={0:sample,1:sample})


# test
# data_t,y_t = mc(n_samples=100,n_features= 5,n_informative=3) #10_000_000
# data_t = pd.DataFrame(data_t)
# y_t = pd.Series(y_t,name='stabf')
# data_t = pd.concat((data_t,y_t),axis=1)
# df_test_spark = spark.createDataFrame(data_t)
df_test_spark = df_train_spark.stat.sampleBy("stabf",fractions={0:sample,1:sample})

# drop val ad test from train
df_train_spark = df_train_spark.subtract(df_test_spark.union(df_val_spark))
# drop y column from test
test_y =  df_test_spark.select('stabf')
df_test_spark = df_test_spark.drop('stabf')



In [0]:
df_train_spark.rdd.getNumPartitions()

Out[102]: 67

In [0]:
# Repartition data
df_train_spark = df_train_spark.repartition(16)
df_val_spark = df_val_spark.repartition(16)
df_test_spark = df_test_spark.repartition(16)

In [0]:
df_train_spark.show()
df_val_spark.show()
df_test_spark.show()

+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-----+
 0| 1| 2| 3| 4| 5| 6| 7| 8| 9|stabf|
+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-----+
 0.18725745949864395| -2.793925812170121| 1.2329461590383006| 5.940636866443798|-0.40940253673433147| 1.621042637011699| 0.7453733526272042| -4.215026898261057| -0.6201737717875699| -1.7787438611455495| 0|
 -1.985033220402021| 1.9643207704344183| 1.7381112296350318|-0.5432687824904533| -0.513920042792523| -3.83105366896641| 3.1916596692505763| 0.7715274877796405| -0.863395445374952| 1.538199832712794| 0|
 0.31414436773683846| 0.10561658913948846| 1.362592944056634| -3.279332685511143| 3.5071844396199423|-0.31171918343427085| -1.0075842267877628|-0.7557286749759665| -0.4214846084124842| 2.679205171068332| 0|
 -4.712494659656528| 1.1280645802453053| 0.03762083902092617| 2.201342833071576| -0.9969603832205323| -1.481632671922279| 1.8642151719226205|-1.2519879014945685| 0.9388818215873439| 4.328475590773305| 0|
 1.865249550668668| -2.346442356665017| -1.5365398318018193| -0.87338152361559| 2.709446484716948| 0.3501377655383472| -2.0019239539173785| 0.8616090572179229|-0.27433471639540835| -1.1744650317962286| 1|
 0.34059020056718403|-0.09288521498695812| 2.289260609725921| -4.387057018495968| 1.086275573138364|-0.31049412836522894| -0.8228395428761031| 2.1976802044851116| 1.1192705131333127| 1.8265792194291155| 0|
 4.234003639542076| 1.09474893782175| 2.3925982982564924| -5.270238479165037| 2.0900521252466175| -1.8907290268753085| -4.110532815070033|-0.1766081898511873| 0.630808238067427| 0.48411250460192345| 0|
 1.5677002341840192| -1.344799980345372| 0.9139643861925086|0.18778352104464377| 2.2011193977323735| 1.102102196361788| -1.879239415080201| -2.585828904494785| 0.9156616578457817| 0.2624429619724906| 1|
 -3.582147778425689| -3.1391260602087754|-0.15277638482438016| 2.3678684415310296| 1.4573509349092157| -2.2660090189465634| -0.7391328051934684|-0.1584515002660013| 0.2100950811953466| 4.202157516240058| 0|
 -1.8970897747242557| 0.3789874417791932| -1.6527663481529942| 2.2247470853450917| 0.7470130133443056| -4.040119658565095| 4.5575113641369445| 1.1782965227745033| 0.5861048170189164| -1.1983072243228028| 0|
 0.9419734656660609| 0.29829402908621394| 2.720272998607312|-2.1373314069656404| 3.12840979972103| 2.6392260595704906| 0.41220127737642365|-2.9369577992369056| 1.487036869115791| 1.272155590758941| 1|
 -1.464305933514512| -1.7661833873882549| -2.1528809135039033| 2.6189722214308| -0.2974402762513013| -3.3460250847722754| 0.44747564656223693| 1.8148941569397308|-0.31376583740718883|-0.32001549876950486| 1|
 1.196554712928476| -0.8808547525099041| 1.9980475918230223| 1.005131044785536| 0.9521946685576085| 1.1653236095494326|-0.20783715821493054|-2.7480709005916086| -2.3351765702006846| -0.4200534889169014| 1|
 0.5896385874303179| -2.3563965011566133| 1.84368775655313| 2.3855334538257633| 1.6820918552415876| -1.6616386530921305| 0.7345874556215239|-1.9224176647765419| 1.642874648107958| -0.6524357026338525| 0|
 -1.6990568625858455| -1.6698596067368943| 1.0780415318289536| 2.3461200925122996| 0.18526459103901993| 0.4386669435699604| 0.20941056761224197|-1.7093617791883475| 1.0587540422299766| 1.7080258156060801| 0|
 2.6671901621369116| 0.2730558533495231| 1.1451788442854942|-1.4010024453298058| 0.6440110062709501| 0.7099519754312679| -1.1765153298824127|-0.8646056528300883|-0.05381489919545...| -1.4579479111946911| 1|
 4.165292188836131|-0.18204743738512652| 5.86762659470372| -5.672419367600697| 1.833766027565276| 3.708329228650273| -4.2611900678339385|-2.2253385537521986| 0.9425205833069585| 1.4603414935788974| 1|
 -2.693169447296832| -0.636328324561706| 0.0759

In [0]:
# check label ratio in each partetion
from typing import Iterator
import pandas as pd

def pandas_filter(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
  for pdf in iterator:
    #df = pd.DataFrame()
    pdf = pd.DataFrame(pdf.groupby('stabf').size()).reset_index().rename({0:"count"},axis=1)
    yield pdf
    
df_train_spark.mapInPandas(pandas_filter,spark.createDataFrame(pd.DataFrame(dict(stabf=[0],count=[0]))).schema).show(40)


+-----+-----+
stabf|count|
+-----+-----+
 0| 5039|
 1| 4961|
 0| 4959|
 1| 5041|
 0| 5041|
 1| 4959|
 0| 5044|
 1| 4956|
 0| 5046|
 1| 4954|
 0| 5007|
 1| 4993|
 0| 4920|
 1| 5080|
 0| 5061|
 1| 4939|
 0| 5009|
 1| 4991|
 0| 5010|
 1| 4990|
 0| 4919|
 1| 5081|
 0| 4926|
 1| 5074|
 0| 4953|
 1| 5047|
 0| 4948|
 1| 5052|
 0| 2067|
 1| 2105|
 0| 4973|
 1| 5027|
 0| 4942|
 1| 5058|
 0| 4996|
 1| 5004|
 0| 5041|
 1| 4959|
 0| 4973|
 1| 5027|
+-----+-----+
only showing top 40 rows

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
from typing import Iterator
from pycaret.classification import *
from sklearn import metrics as m
import pickle
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
import time
from pyspark.ml.evaluation import MulticlassClassificationEvaluator as mult
from sklearn.metrics import roc_auc_score

class Binary_Classifier():
  
  def __init__(self,data:DataFrame,
    target_variable:str,
    validation_percentage:float=0.1,
    validation_data:DataFrame =None, 
    batch_size:int =3,
    prediction_threshold:float=0.50,
    session_id:int=None,
    gpu_training:bool=False,
    stratified=False,
    _count=True):
    
    """
    Docs String Here
    """
    
    self.training_data = data
    self.target_variable = target_variable
    self.batch_size= batch_size
    self.validation_data = validation_data
    self.validation_percentage = validation_percentage
    self.prediction_threshold = prediction_threshold
    self.gpu_training = gpu_training
    self.session_id = session_id
    self.validation_data = validation_data
    self._data_after_preprocessing = None
    self.schema = self.training_data.schema
    self.stratified = stratified
    self._count = _count
    
  
  def select_model(self,model_name:str,**custom_parameters):
    """
    Doc String
    """
    self.model_name = model_name
    self.custom_parameters = custom_parameters
    
    
  def auto_select_model(self):
    """
    Doc String
    """
    self.model_name = "Auto Select"

    
  
  def fit_predict(self,blend=False):
    
    self.blend_option = blend
    
    # make/check tarin & validate data
    #=================================
    # future proprocessign step
    
    
    
    if self.validation_data:
      self.validation_data = self.validation_data
    else:
      self.validation_data = self.training_data.stat.sampleBy("stabf",{0:self.validation_percentage,1:self.validation_percentage},seed=i) # hard coded - labels & seed
      # remove from training
      self.training_data = self.training_data.subtract(self.validation_data)
      
    
    # if user wants to stratify:
    #make samples
    #============
    if self.stratified:
      start = time.perf_counter()
      fraction = 1/self.batch_size
      # ph
      df_sample = spark.createDataFrame(data=[],schema=self.training_data.schema)
      df_sample = df_sample.withColumn("sample_index",lit(0))
      self.training_data= self.training_data.withColumn("sample_index",lit(0))
      self.group_column = "sample_index"

      for i in range(self.batch_size):
        sample = self.training_data.stat.sampleBy("stabf",{0:fraction,1:fraction},seed=i) # hard coded - labels & seed
        sample = sample.withColumn("sample_index",lit(i))
        df_sample = df_sample.union(sample)

      # save the sampled data
      self.df_sample = df_sample
      if self._count:
        self.df_sample.count()
      
      end = time.perf_counter()
      print("sampling took {0} minutes to run ...".format((end-start)/60))
    else:
      # Else , rename training data to sample data
      self.df_sample =  self.training_data 
    
    
    #Training output Scheema
    #=======================
    
    self.internal_training_schema = StructType([StructField("model_id", LongType(), True),  
                       StructField("model_returend", BinaryType(), True),
                       StructField("Accuracy",FloatType(), True),
                       StructField("AUC",FloatType(), True)])
  
    
    # make training function
    #======================
    # this function will not take class attributes , so we will need to give it local/enclosing variables
    if self.stratified:
      group_column = self.group_column
    else:
      group_column= 'na'
    
    target = self.target_variable
    sessionid = self.session_id
    gpu = self.gpu_training
    model_name = self.model_name
    
    # if custome model only then make cp
    if model_name != "Auto Select":
      cp_create_model = self.custom_parameters
    
    # if user wants stratify the we do the apply in pandas way
    if self.stratified:
      start = time.perf_counter()
      def internal_training(key,pdf):
        # get the sample id
        id = pdf[group_column].iloc[0]
        # create a smaller dataset
        #small = pdf.head(10000)
        #call setup
        #cls1= setup(small.drop([group_column],axis=1), target = target, session_id=sessionid,silent=True,html=False,preprocess=False,use_gpu=gpu)
        cls1= setup(pdf.drop([group_column],axis=1), target = target, session_id=sessionid,silent=True,html=False,preprocess=False,use_gpu=gpu)
        # Now change the it with original (since we do not need preprocessing)
        #set_config('X_train',pdf.drop([group_column,target],axis=1))
        #set_config('y_train',pdf[target])

        #Modeling
        # if create model is selected
        if model_name == "Auto Select":
          model = compare_models(n_select=1,cross_validation=False) 
        else:
          model = create_model(model_name ,cross_validation=False,**cp_create_model) # while unpacking the custom parameters



        # try to get AUC
        try:
          prediction_prob = model.predict_proba (pdf.drop([group_column,target],axis=1))
          AUC = m.roc_auc_score(pdf[target],prediction_prob[:, 1])
        except:
          AUC = 0.0


        # get prediction & accuracy
        prediction = model.predict(pdf.drop([group_column,target],axis=1))  
        accuracy = m.accuracy_score(y_true=pdf[target],y_pred=prediction)
        # convert to df
        return_df = pd.DataFrame(dict(model_id = id,model_returend = [pickle.dumps(model)], Accuracy = accuracy, AUC = AUC))


        return(return_df)


      # Call the training function with sampled data
      #=============================================
      return_internal_training  = self.df_sample.groupby(self.group_column).applyInPandas(internal_training, schema=self.internal_training_schema)
      return_internal_training.cache()
      if self._count:
        return_internal_training.count()
      end = time.perf_counter()
      print("internal training took {0} minutes to run ...".format((end-start)/60))
      return_internal_training.cache()
      self.internal_training_results = return_internal_training
    
    # if user do not want to stratify  
    else:
      
      start = time.perf_counter()
      def internal_training(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:  
        for pdf in iterator:
          # create a smaller dataset
          #small = pdf.head(10000)
          #call setup
          #cls1= setup(small, target = target, session_id=sessionid,silent=True,html=False,preprocess=False,use_gpu=gpu)
          cls1= setup(pdf, target = target, session_id=sessionid,silent=True,html=False,preprocess=False,use_gpu=gpu)
          # Now change the it with original (since we do not need preprocessing)
          #set_config('X_train',pdf.drop([target],axis=1))
          #set_config('y_train',pdf[target])

          #Modeling
          # if create model is selected
          if model_name == "Auto Select":
            model = compare_models(n_select=1,cross_validation=False) 
          else:
            model = create_model(model_name ,cross_validation=False,**cp_create_model) # while unpacking the custom parameters

          # try to get AUC
          try:
            prediction_prob = model.predict_proba (pdf.drop([target],axis=1))
            AUC = m.roc_auc_score(pdf[target],prediction_prob[:, 1])
          except:
            AUC = 0.0

          # get prediction & accuracy
          prediction = model.predict(pdf.drop([target],axis=1))  
          accuracy = m.accuracy_score(y_true=pdf[target],y_pred=prediction)
          
          # convert to df, give same name as itreator element
          # for now we will keep model id as 0 , alter we will fill it by inremental index
          pdf = pd.DataFrame(dict(model_id = 0,model_returend = [pickle.dumps(model)], Accuracy = accuracy, AUC = AUC))
          # retuen this portion 
          yield pdf
      
      # Call the training function with sampled data
      #=============================================
      return_internal_training  = self.df_sample.mapInPandas(internal_training, schema=self.internal_training_schema)
      # extra step to fill the model_id column with increasing ID numbers
      return_internal_training = return_internal_training.withColumn("model_id",monotonically_increasing_id())
      window = Window.orderBy(col('model_id'))
      return_internal_training=return_internal_training.withColumn('model_id', row_number().over(window))
      return_internal_training = return_internal_training.withColumn('model_id',expr("model_id - 1"))
      return_internal_training.cache()
      
      # id is not unique index 
      if self._count:
        return_internal_training.count()
      end = time.perf_counter()
      print("internal training took {0} minutes to run ...".format((end-start)/60))

      self.internal_training_results = return_internal_training
      
      

    # Collect trained models in a dic
    #================================
    start = time.perf_counter()
    dict_models ={}
    for r in return_internal_training.rdd.collect():
      dict_models[r['model_id']] = pickle.loads(r['model_returend'])
    end = time.perf_counter()
    print("collecting models took {0} minutes to run ...".format((end-start)/60))
    spark.sparkContext.broadcast(dict_models)
    self.internal_trained_models = dict_models
    
    # Apply the second Level training 
    #================================
    #================================
    
    
    # First, creat a schema for training
    # outcome (predictions)
    #==================================
    
    # this has to be flexable, depending upon the 
    # batches we made 
    # easiest way is to creat a dummy pandas df 
    # with required column name and data types (always float)
    dummy = pd.DataFrame(dict(zip (list(range(self.batch_size)) , [[i*1.0,i*1.0] for i in range(self.batch_size)])))
    # add one extra column named labels
    dummy['label'] = 0.0
    # get the schema
    schema2 = spark.createDataFrame(dummy).schema
    self._useful_schema =  spark.createDataFrame(dummy).schema
    
    # creat another pandas udf 
    # for generating predictions
    # on entire training  data set
    #===========================
    # this will not take any instances
    def training_prediction_all(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
      for pdf in iterator:
        res = pd.DataFrame()
        for k in dict_models.keys():
          try:
            pre = dict_models[k].predict_proba(pdf.drop([group_column,target],axis=1,errors='ignore'))
            res[str(k)] = pre[:, 1]
          except:
            pre = dict_models[k].predict(pdf.drop([group_column,target],axis=1,errors='ignore'))
            res[str(k)] = pre

        # put the prediction with the results
        res['label'] = pdf[target]
        yield (res)
    
    start = time.perf_counter()
    return_internal_training_second = self.df_sample.mapInPandas(training_prediction_all, schema=schema2)
    if self._count:
      return_internal_training_second.count()
    end = time.perf_counter()
    
    print("predicion on entire training data took {0} minutes to run ...".format((end-start)/60))
    
    #Validation Predictionson
    # ========================
    #return predictions on entire validation data 
    # Do we need this ? remmeber we have OOB predictions from training data
    def validation_prediction_all(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
      for pdf in iterator:
        res = pd.DataFrame()
        for k in dict_models.keys():
          try:
            pre = dict_models[k].predict_proba(pdf.drop([group_column,target],axis=1,errors='ignore'))
            res[str(k)] = pre[:, 1]
          except:
            pre = dict_models[k].predict(pdf.drop([group_column,target],axis=1,errors="ignore"))
            res[str(k)] = pre

        # put the prediction with the results
        res['label'] = pdf[target]
        yield (res)
    
    start = time.perf_counter()
    return_internal_validation_second = self.validation_data.mapInPandas(validation_prediction_all, schema=schema2)
    if self._count:
      return_internal_validation_second.count()
    end = time.perf_counter()
    print("predicion on entire validation data took {0} minutes to run ...".format((end-start)/60))
    
    #Now train on second dataset with spark
    # # only IF Blend is set to True
    # So far only selecting lr
    #=======================================
    if blend:
      mlib_lr = LogisticRegression()
      # make feature vectors
      self.vector_assembler = VectorAssembler(inputCols=[str(i) for i in range(self.batch_size)],outputCol='features')
      # now transform
      df_training = self.vector_assembler.transform(return_internal_training_second)
      df_validation = self.vector_assembler.transform(return_internal_validation_second)

      # only select feature vector and target column

      df_training = df_training.select("features","label")


      df_validation = df_validation.select("features","label")

      # now train
      start = time.perf_counter()
      self.spark_fitted_model = mlib_lr.fit(df_training)
      end = time.perf_counter()
      print("spark model fitting took {0} minutes to run ...".format((end-start)/60))

      start = time.perf_counter()    
      Evaluated = self.spark_fitted_model.evaluate(df_validation)
      end = time.perf_counter()
      print("spark model evaluation took {0} minutes to run ...".format((end-start)/60))

      # make a data frame
      start = time.perf_counter()
      results = pd.DataFrame(dict(Accuracy=[self.spark_fitted_model.summary.accuracy,Evaluated.accuracy],AUC=[self.spark_fitted_model.summary.areaUnderROC,Evaluated.areaUnderROC]),index=['Training','Validation'])
      end = time.perf_counter()
      print("spark results took {0} minutes to run ...".format((end-start)/60))

      #self.train_res = fitted_model
      self.val_res = Evaluated
      
      
      
    # # only IF Blend is set to False
    # Then simple average with soft voting
    # So far only selecting lr
    #=======================================
    if not self.blend_option:
      
      # FOR TRAINING DATA SET
      # we need the table plus two more columns (score & voted prediction)
      return_internal_training_second = return_internal_training_second.withColumn("Average_Score",lit(0.0))
      return_internal_validation_second = return_internal_validation_second.withColumn("Average_Score",lit(0.0))
      
      l = self.batch_size
      cols = return_internal_training_second.columns[:-3]
      self._useful_cols = return_internal_training_second.columns[:-3]
      def pandas_filter(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
          for pdf in iterator:
            pdf['Average_Score'] = pdf[cols].mean(axis=1)
            yield pdf
      
      # For Training
      #-------------
      start = time.perf_counter()
      training_predictions  = return_internal_training_second.mapInPandas(pandas_filter, schema=return_internal_training_second.schema)
      # get prediction logic
      training_predictions = training_predictions.withColumn("prediction",when(training_predictions.Average_Score>=self.prediction_threshold,1.0).otherwise(0.0))
      self.training_predictions = training_predictions
      if self._count:
        self.training_predictions.count()
      end = time.perf_counter()
      print("voting on training data took {0} minutes to run ...".format((end-start)/60))
      
      
      # For Validation
      #---------------
      start = time.perf_counter()
      validation_predictions  = return_internal_validation_second.mapInPandas(pandas_filter, schema=return_internal_validation_second.schema)
      # get prediction logic
      validation_predictions = validation_predictions.withColumn("prediction",when(validation_predictions.Average_Score>=self.prediction_threshold,1.0).otherwise(0.0))
      self.validation_predictions = validation_predictions
      if self._count:
        self.validation_predictions.count()
      end = time.perf_counter()
      print("voting on validation data took {0} minutes to run ...".format((end-start)/60))
      
      # Accuracy, recall, precision , F1 will be calculated through multicalss evaluator 
      # AUC will be calculated through sklearn 
      training_accuracy = mult(predictionCol='prediction',probabilityCol='Average_Score',metricName='accuracy').evaluate(self.training_predictions)
      validation_accuracy =  mult(predictionCol='prediction',probabilityCol='Average_Score',metricName='accuracy').evaluate(self.validation_predictions)
      # define AUC function
      
      #@pandas_udf("double")
      #def pandas_auc(label: pd.Series,probability: pd.Series) -> float:
      #  score = roc_auc_score(y_true= label, y_score=probability)
      #  return score
      
      #training_auc = self.training_predictions.select(pandas_auc('label','Average_Score')).first()[0]
      #validation_auc = self.validation_predictions.select(pandas_auc('label','Average_Score')).first()[0]
      
      sch = spark.createDataFrame(pd.DataFrame(dict(auc=[0.0]))).schema
      def pandas_auc(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
        for pdf in iterator:
          auc = m.roc_auc_score(pdf.label,pdf.Average_Score)
          report = pd.DataFrame(dict(auc=[auc]))
          yield report

      training_auc_table = self.training_predictions.mapInPandas(pandas_auc, schema=sch)
      validation_auc_table = self.validation_predictions.mapInPandas(pandas_auc, schema=sch)
      
      # Now take the median
      training_auc = training_auc_table.approxQuantile('auc',[0.50],0)[0]
      validation_auc = validation_auc_table.approxQuantile('auc',[0.50],0)[0]
      
      
      #m_precision = m(predictionCol='Prediction',probabilityCol='Average_score',metricName='precisionByLabel').evaluate((cls.training_predictions))
      #recall=  m(predictionCol='Prediction',probabilityCol='Average_score',metricName='weightedRecall').evaluate((cls.training_predictions))
      start = time.perf_counter()
      results = pd.DataFrame(dict(Accuracy=[training_accuracy,validation_accuracy],AUC=[training_auc,validation_auc]),index=['Training','Validation'])
      end = time.perf_counter()
      print("results took {0} minutes to run ...".format((end-start)/60))
      
    
        
    return(results)
    
    
  #predict functio
  def predict(self,data:DataFrame):
    
    # Predictionson with Internal Trained Models
    # ========================
    #return predictions on entire validation data 
    # Do we need this ? remmeber we have OOB predictions from training data
    dict_models = self.internal_trained_models
    def test_prediction_all(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
      for pdf in iterator:
        res = pd.DataFrame()
        for k in dict_models.keys():
          try:
            pre = dict_models[k].predict_proba(pdf)
            res[str(k)] = pre[:, 1]
          except:
            pre = dict_models[k].predict(pdf)
            res[str(k)] = pre

        # put the prediction with the results
        #res['label'] = pdf[target]
        yield (res)
    
    start = time.perf_counter()
    return_internal_test_second = data.mapInPandas(test_prediction_all, schema=self._useful_schema[:-1])# dont need the lable column, remove last
    if self._count:
      return_internal_test_second.count()
    end = time.perf_counter()
    print("predicion on entire validation data took {0} minutes to run ...".format((end-start)/60))
    
    
    #Final Predictions
    #=================
    # Without Blending
    #-----------------
    
    # # only IF Blend is set to False
    # Then simple average with soft voting
    # So far only selecting lr
    #=======================================
    if not self.blend_option:
      
      # we need the table plus two more columns (score & voted prediction)
      return_internal_test_second = return_internal_test_second.withColumn("Average_Score",lit(0.0))
      
      
      l = self.batch_size
      cols = self._useful_cols
      def pandas_predict(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
          for pdf in iterator:
            pdf['Average_Score'] = pdf[cols].mean(axis=1)
            yield pdf
      

      # For Validation
      #---------------
      start = time.perf_counter()
      test_predictions  = return_internal_test_second.mapInPandas(pandas_predict, schema=return_internal_test_second.schema)
      # get prediction logic
      test_predictions = test_predictions.withColumn("Prediction",when(test_predictions.Average_Score>=self.prediction_threshold,1.0).otherwise(0.0))
      self.test_predictions = test_predictions
      if self._count:
        self.test_predictions.count()
      end = time.perf_counter()
      print("voting on validation data took {0} minutes to run ...".format((end-start)/60))
      
      # put the predictions with the test data
      
      # add id Column to data  and predictions
      transformed = data.withColumn("sudo_id",monotonically_increasing_id())
      test_predictions = test_predictions.withColumn("sudo_id",monotonically_increasing_id())
      transformed = transformed.join(test_predictions.select("sudo_id","Average_Score","Prediction"),"sudo_id",'inner')
      transformed = transformed.sort("sudo_id")
      transformed = transformed.drop('sudo_id')
       
      
   # Without Blending
    #-----------------
    if self.blend_option :
      
      
      # get features through vector_assembler
      df_test = self.vector_assembler.transform(return_internal_test_second)

      # only select feature vector and target column
      #df_test = df_test.select("features")

      start = time.perf_counter()    
      transformed = self.spark_fitted_model.transform(df_test)
      # drop features and raw_prediction columns
      transformed = transformed.drop('features','rawPrediction')
      end = time.perf_counter()
      print("spark model evaluation took {0} minutes to run ...".format((end-start)/60))
      
        
    return(transformed)
    
  

In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=16,gpu_training=False,validation_data=df_val_spark,stratified=False,_count=False)
cls.select_model('lr',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

internal training took 0.0009196958666658853 minutes to run ...
collecting models took 0.0009251358999942265 minutes to run ...
predicion on entire training data took 0.0010371516166590784 minutes to run ...
predicion on entire validation data took 0.0008967907166758474 minutes to run ...
voting on training data took 0.003258654116658969 minutes to run ...
voting on validation data took 0.0007394163833244723 minutes to run ...
results took 1.283175000329114e-05 minutes to run ...
Out[37]:

,Accuracy,AUC
Training,0.705689,0.775683
Validation,0.704779,0.774596


In [0]:
cls.training_predictions.select('label','Average_Score','prediction').show()
pred = cls.training_predictions.toPandas()

+-----+-------------------+----------+
label| Average_Score|prediction|
+-----+-------------------+----------+
 0.0| 0.5511077030753134| 1.0|
 0.0| 0.7645165422198162| 1.0|
 0.0|0.09059323313077358| 0.0|
 0.0| 0.3231501429846937| 0.0|
 0.0| 0.7687786801084281| 1.0|
 0.0|0.23317629059219086| 0.0|
 1.0| 0.6710602713178658| 1.0|
 1.0|0.21794474263983352| 0.0|
 0.0|0.13414946458096574| 0.0|
 1.0| 0.2575282232237377| 0.0|
 1.0| 0.7870038368423795| 1.0|
 1.0|0.33802131406549224| 0.0|
 0.0| 0.7527687817073592| 1.0|
 0.0| 0.6315110498324416| 1.0|
 1.0| 0.8518262540749661| 1.0|
 1.0| 0.7946356196710854| 1.0|
 1.0|0.42826740274970093| 0.0|
 0.0| 0.1794023034931792| 0.0|
 0.0|0.02240768381351055| 0.0|
 1.0| 0.5504248563974711| 1.0|
+-----+-------------------+----------+
only showing top 20 rows

In [0]:
m.roc_auc_score(pred['label'],pred.Average_Score)

Out[14]: 0.7736574915232922

In [0]:
sc = spark.createDataFrame(pd.DataFrame(dict(auc=[0.0]))).schema
from typing import Iterator
import pandas as pd

def roc_auc(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
  for pdf in iterator:
    auc = m.roc_auc_score(pdf.label,pdf.Average_Score)
    report = pd.DataFrame(dict(auc=[auc]))
    yield report

pred_spark = cls.training_predictions.mapInPandas(roc_auc, schema=sc)
pred_spark.show()

+------------------+
 auc|
+------------------+
0.7814300133238161|
0.7671693517017543|
 0.772778213316146|
0.7729395249869632|
0.7660434096401582|
 0.779520440603806|
0.6121212121212122|
0.7748195265941952|
0.7674492662316559|
0.7729878924102602|
 0.775034633328745|
0.7712788589677405|
 0.77216528|
0.6282051282051282|
0.7686902706238352|
0.7833699817086127|
0.7675418305906431|
 0.773403326264585|
0.7758283992982238|
0.7787744690635179|
+------------------+
only showing top 20 rows

In [0]:
# from pyspark.sql.functions
pred_spark.approxQuantile('auc',[0.50],0)

Out[30]: [0.7736697275339937]

In [0]:
cls.training_predictions.schema

Out[16]: StructType(List(StructField(0,DoubleType,true),StructField(1,DoubleType,true),StructField(2,DoubleType,true),StructField(3,DoubleType,true),StructField(4,DoubleType,true),StructField(5,DoubleType,true),StructField(6,DoubleType,true),StructField(7,DoubleType,true),StructField(8,DoubleType,true),StructField(9,DoubleType,true),StructField(10,DoubleType,true),StructField(11,DoubleType,true),StructField(12,DoubleType,true),StructField(13,DoubleType,true),StructField(14,DoubleType,true),StructField(15,DoubleType,true),StructField(label,DoubleType,true),StructField(Average_Score,DoubleType,false),StructField(prediction,DoubleType,false)))

In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=32,gpu_training=False,validation_data=df_val_spark,stratified=True,_count=False)
cls.select_model('lr',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

sampling took 0.01638848868333298 minutes to run ...
internal training took 0.11348518975000843 minutes to run ...
collecting models took 0.637840536699999 minutes to run ...
predicion on entire training data took 0.0008705557666871755 minutes to run ...
predicion on entire validation data took 0.00067376391665069 minutes to run ...
voting on training data took 0.0010591287833449313 minutes to run ...
voting on validation data took 0.0008102054666778713 minutes to run ...
results took 1.4950900003896095e-05 minutes to run ...
Out[97]:

,Accuracy,AUC
Training,0.785893,0.831813
Validation,0.786128,0.831959


In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=32,gpu_training=False,validation_data=df_val_spark,stratified=False,_count=False)
cls.select_model('lr',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

internal training took 0.0009155517999867394 minutes to run ...
collecting models took 0.5549314870666573 minutes to run ...
predicion on entire training data took 0.014675810333331658 minutes to run ...
predicion on entire validation data took 0.0091665485999935 minutes to run ...
voting on training data took 0.0008349569499993474 minutes to run ...
voting on validation data took 0.0007970580000043507 minutes to run ...
results took 1.448921666451497e-05 minutes to run ...
Out[58]:

,Accuracy,AUC
Training,0.905806,0.946371
Validation,0.905971,0.946611


In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=32,gpu_training=False,validation_data=df_val_spark,stratified=False,_count=True)
cls.select_model('lr',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

internal training took 0.4826428809999925 minutes to run ...
collecting models took 0.43155068553332965 minutes to run ...
predicion on entire training data took 1.8638787871000053 minutes to run ...
predicion on entire validation data took 0.6177722858999914 minutes to run ...
voting on training data took 1.7799080259166657 minutes to run ...
voting on validation data took 0.5886574604000089 minutes to run ...
results took 0.00023758929999833828 minutes to run ...
Out[42]:

,Accuracy,AUC
Training,0.460334,0.510785
Validation,0.460330,0.510654


In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=32,gpu_training=False,validation_data=df_val_spark,stratified=False)
cls.select_model('lr',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

internal training took 0.3407781568499862 minutes to run ...
collecting models took 0.2874203822666762 minutes to run ...
predicion on entire training data took 0.7584421598833312 minutes to run ...
predicion on entire validation data took 0.25094252286665625 minutes to run ...
voting on training data took 0.7858218102500056 minutes to run ...
voting on validation data took 0.2524143038333326 minutes to run ...
results took 1.2636750003972944e-05 minutes to run ...
Out[8]:

,Accuracy,AUC
Training,0.461152,0.470598
Validation,0.460187,0.469867


In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=32,gpu_training=False,validation_data=df_val_spark,stratified=False)
cls.select_model('lr',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

internal training took 0.36030952461669585 minutes to run ...
collecting models took 0.3152949938000044 minutes to run ...
predicion on entire training data took 0.7662629161333219 minutes to run ...
predicion on entire validation data took 0.2510455733000223 minutes to run ...
voting on training data took 0.790862216300017 minutes to run ...
voting on validation data took 0.25284859646665914 minutes to run ...
results took 1.3246766654143964e-05 minutes to run ...
Out[44]:

,Accuracy,AUC
Training,0.839871,0.882626
Validation,0.839803,0.882678


In [0]:
cls.batch_size

Out[34]: 3

In [0]:
cls.internal_trained_models

Out[32]: {0: LogisticRegression(max_iter=1000, random_state=32),
 8589934592: LogisticRegression(max_iter=1000, random_state=32),
 17179869184: LogisticRegression(max_iter=1000, random_state=32),
 25769803776: LogisticRegression(max_iter=1000, random_state=32),
 34359738368: LogisticRegression(max_iter=1000, random_state=32),
 42949672960: LogisticRegression(max_iter=1000, random_state=32),
 51539607552: LogisticRegression(max_iter=1000, random_state=32),
 60129542144: LogisticRegression(max_iter=1000, random_state=32),
 68719476736: LogisticRegression(max_iter=1000, random_state=32),
 77309411328: LogisticRegression(max_iter=1000, random_state=32),
 85899345920: LogisticRegression(max_iter=1000, random_state=32),
 94489280512: LogisticRegression(max_iter=1000, random_state=32),
 103079215104: LogisticRegression(max_iter=1000, random_state=32),
 111669149696: LogisticRegression(max_iter=1000, random_state=32),
 120259084288: LogisticRegression(max_iter=1000, random_state=32),
 128849018880: LogisticRegression(max_iter=1000, random_state=32),
 137438953472: LogisticRegression(max_iter=1000, random_state=32),
 146028888064: LogisticRegression(max_iter=1000, random_state=32),
 154618822656: LogisticRegression(max_iter=1000, random_state=32),
 163208757248: LogisticRegression(max_iter=1000, random_state=32),
 171798691840: LogisticRegression(max_iter=1000, random_state=32),
 180388626432: LogisticRegression(max_iter=1000, random_state=32),
 188978561024: LogisticRegression(max_iter=1000, random_state=32),
 197568495616: LogisticRegression(max_iter=1000, random_state=32),
 206158430208: LogisticRegression(max_iter=1000, random_state=32),
 214748364800: LogisticRegression(max_iter=1000, random_state=32),
 223338299392: LogisticRegression(max_iter=1000, random_state=32),
 231928233984: LogisticRegression(max_iter=1000, random_state=32),
 240518168576: LogisticRegression(max_iter=1000, random_state=32),
 249108103168: LogisticRegression(max_iter=1000, random_state=32),
 257698037760: LogisticRegression(max_iter=1000, random_state=32),
 266287972352: LogisticRegression(max_iter=1000, random_state=32)}

In [0]:
cls.internal_training_res.select("model_id",'Accuracy').toPandas().head()

Out[36]:

,model_id,Accuracy
0,0,0.989950
1,8589934592,0.979900
2,17179869184,0.989950
3,25769803776,0.994975
4,34359738368,0.989950


In [0]:
monotonically_increasing_id()

In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=3,gpu_training=False,validation_data=df_val_spark)
cls.select_model('lr',random_state=32)
# cls.auto_select_model()
cls0 = cls.fit_predict(True)

processing took 0.02464658870000373 minutes to run ...
internal training took 0.07698263590000351 minutes to run ...
collecting models took 0.07947604143332683 minutes to run ...
predicion on entire training data took 0.05002794574999522 minutes to run ...
predicion on entire validation data took 0.003980531016668465 minutes to run ...
spark model fitting took 0.27660734771666284 minutes to run ...
spark model evaluation took 0.0026070444166634843 minutes to run ...
spark results took 0.08778485640000326 minutes to run ...

In [0]:
cls0

Out[9]: ( Accuracy AUC
 Training 0.886579 0.940654
 Validation 0.882412 0.933407,
 <__main__.Binary_Classifier at 0x7fa788f9c390>)

In [0]:
df = spark.createDataFrame(cls0[0])

In [0]:
# save and testing
filename = '/dbfs/FileStore/Fahad/SDM_Cash_Rec/cls_object.sav'
pickle.dump(spark.createDataFrame(cls0[0]), open(filename, 'wb'))

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1899558221411698> in <module> 
 1 # save and testing 
 2 filename = '/dbfs/FileStore/Fahad/SDM_Cash_Rec/cls_object.sav' 
 ----> 3 pickle . dump ( spark . createDataFrame ( cls0 [ 0 ] ) , open ( filename , 'wb' ) ) 

 TypeError : can't pickle _thread.RLock objects

In [0]:
import pickle
file_path= '/dbfs/FileStore/Fahad/SDM_Cash_Rec/cls_object.sav'
loaded_cls1  = pickle.load(open(file_path,'rb'))

In [0]:
loaded_cls1

Out[3]:

,Accuracy,AUC
Training,0.708912,0.809452
Validation,0.713202,0.816464


In [0]:
test_res = cls.predict(df_test_spark)
df_test = test_res.toPandas()
df_test['label']=test_y.toPandas()
# print(m.accuracy_score(df_test.label,df_test.Prediction))
# print(m.roc_auc_score(df_test.label,df_test.Average_Score))
print(m.accuracy_score(df_test.label,df_test.prediction))
# print(m.roc_auc_score(df_test.label,df_test.Average_Score))

predicion on entire validation data took 0.002900260416633197 minutes to run ...
spark model evaluation took 0.0004659662833243298 minutes to run ...
0.9212481426448736

In [0]:
test_res.show(truncate=False)

+--------------------+--------------------+---------------------+-----------------------------------------+----------+
0 |1 |2 |probability |prediction|
+--------------------+--------------------+---------------------+-----------------------------------------+----------+
0.06814960199456437 |0.062062449749684244|0.0642814716807269 |[0.9521222480122125,0.04787775198778744] |0.0 |
0.22357376505062337 |0.2548423757348793 |0.18939034758132395 |[0.8225769447694008,0.17742305523059912] |0.0 |
0.11724657653683926 |0.12254127448041349 |0.12838447352336343 |[0.9361187932883314,0.0638812067116686] |0.0 |
0.969273912346278 |0.972970695159838 |0.9721748370555312 |[0.022691921724226904,0.977308078275773] |1.0 |
0.35857780739081585 |0.3827573611472582 |0.40889402741569286 |[0.7439116093481981,0.25608839065180194] |0.0 |
0.5423881513140729 |0.5869222618459158 |0.602046499730448 |[0.4230920741230542,0.5769079258769458] |1.0 |
0.08660765350910966 |0.09461143000963615 |0.08226033100442426 |[0.9432648092230894,0.056735190776910485]|0.0 |
0.9881027897467192 |0.9907476376437344 |0.9868246471687523 |[0.019311763580121464,0.9806882364198786]|1.0 |
0.005928869351028524|0.004207494959058896|0.003893602395578612 |[0.9694475958291575,0.030552404170842425]|0.0 |
0.002058034959497732|0.001529165584404463|0.0012469962101372327|[0.9704259560212977,0.02957404397870231] |0.0 |
0.7822408353127244 |0.7885564689796651 |0.7626739477400313 |[0.07549995801445285,0.9245000419855472] |1.0 |
0.014274726273495578|0.014598060612372902|0.010441112967502457 |[0.9670074476997602,0.032992552300239765]|0.0 |
0.08816332213126703 |0.10530720241436355 |0.10020558554674515 |[0.946715298666014,0.053284701333985905] |0.0 |
7.76770822540236E-4 |5.262242114110947E-4|3.739821908976481E-4 |[0.9707523759543069,0.029247624045693213]|0.0 |
0.05524455703024555 |0.05896691323168656 |0.052534000407581376 |[0.9555595721286434,0.04444042787135652] |0.0 |
0.9039216017563717 |0.9071161099787036 |0.9090072002083893 |[0.037079896416011913,0.9629201035839882]|1.0 |
0.01648127926534245 |0.019910441217463964|0.023792657385017128 |[0.9684067359050688,0.031593264094931274]|0.0 |
0.5322297827004051 |0.5189457979637869 |0.5209714026451694 |[0.37212816960813516,0.6278718303918649] |1.0 |
0.014330355028907903|0.014429529182569303|0.01630688154652937 |[0.9681513213370433,0.03184867866295666] |0.0 |
0.006576719763497682|0.007258337136161985|0.006402219632418821 |[0.9694585698915159,0.030541430108484067]|0.0 |
+--------------------+--------------------+---------------------+-----------------------------------------+----------+
only showing top 20 rows

In [0]:
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=3,gpu_training=False,validation_data=df_val_spark)
cls.select_model('lr',random_state=32)
# cls.auto_select_model()
cls.fit_predict(True)

processing took 0.05109494398332875 minutes to run ...
internal training took 0.15034073899999686 minutes to run ...
collecting models took 0.14040416316665869 minutes to run ...
predicion on entire training data took 0.10709974268332492 minutes to run ...
predicion on entire validation data took 0.004014881366674672 minutes to run ...
spark model fitting took 0.47986135274998865 minutes to run ...
spark model evaluation took 0.0029747748333344743 minutes to run ...
spark results took 0.11969537346667494 minutes to run ...
Out[57]:

,Accuracy,AUC
Training,0.883362,0.930285
Validation,0.880435,0.928343


In [0]:
#median
cls = Binary_Classifier(df_train_spark,'stabf',batch_size=3,gpu_training=False,validation_data=df_val_spark)
cls.select_model('lr',random_state=32)
# cls.auto_select_model()
cls.fit_predict()

processing took 0.07779683614999158 minutes to run ...
internal training took 0.15056046298332149 minutes to run ...
collecting models took 0.14243118601665022 minutes to run ...
predicion on entire training data took 0.11349948890001542 minutes to run ...
predicion on entire validation data took 0.004123904016675321 minutes to run ...
voting on training data took 0.09284943183332264 minutes to run ...
voting on validation data took 0.006449986016650655 minutes to run ...
results took 1.3410133336340853e-05 minutes to run ...
Out[60]:

,Accuracy,AUC
Training,0.878996,0.930967
Validation,0.875988,0.927848


In [0]:
cls.training_predictions.show()

+--------------------+--------------------+--------------------+-----+--------------------+----------+
 0| 1| 2|label| Average_Score|Prediction|
+--------------------+--------------------+--------------------+-----+--------------------+----------+
 0.11319543238246707|0.047430585450009714| 0.08493998155491779| 1.0| 0.0803130089162384| 0.0|
 0.390911313283804| 0.5120969737515351| 0.5411075481046738| 0.0| 0.45150414351766954| 0.0|
 0.08826070231514378| 0.13422089877263726| 0.14956248708402833| 0.0| 0.11124080054389052| 0.0|
2.002907803035254...|4.337383460126931E-4| 7.43520748502953E-4| 0.0|3.170145631581092...| 0.0|
 0.8977683020786055| 0.9176479411263441| 0.8371738756663051| 1.0| 0.9077081216024747| 1.0|
 0.9980908049922761| 0.9970895049611381| 0.9957063402782154| 1.0| 0.9975901549767071| 1.0|
0.007474586160136...|0.006148575452990653|0.004747447704471...| 0.0|0.006811580806563743| 0.0|
 0.19478340831849758| 0.3068936560336814| 0.18775863175039087| 0.0| 0.2508385321760895| 0.0|
 0.9971327566202993| 0.995733589814662| 0.9967490905361177| 1.0| 0.9964331732174807| 1.0|
 0.9994895358285135| 0.9995098762225537| 0.9983513899123314| 1.0| 0.9994997060255336| 1.0|
 0.08075697630221325| 0.08559967687931998| 0.04696116614032551| 1.0| 0.08317832659076661| 0.0|
 0.0785664727214567| 0.12599050681995674| 0.0409042889632055| 0.0| 0.10227848977070672| 0.0|
 0.10703640434558839| 0.07845284425385196| 0.10617537408052831| 1.0| 0.09274462429972018| 0.0|
 0.7109049017883965| 0.658989393901947| 0.6223172062336834| 1.0| 0.6849471478451717| 1.0|
0.016015602059205443| 0.02262535226200782|0.019744128714836412| 0.0| 0.01932047716060663| 0.0|
 0.07151923556495614| 0.18369311621948287| 0.08487012761480178| 0.0| 0.1276061758922195| 0.0|
0.006839787567639434| 0.0178036619252292|0.004314487599446156| 0.0|0.012321724746434316| 0.0|
 0.16142769535672713| 0.3513512188122141| 0.14828591399647853| 0.0| 0.2563894570844706| 0.0|
 0.06864335270364734| 0.2163382036547347| 0.11300164260966326| 0.0| 0.142490778179191| 0.0|
 0.10232727346842417| 0.17700869267548294| 0.17660963674573704| 0.0| 0.13966798307195355| 0.0|
+--------------------+--------------------+--------------------+-----+--------------------+----------+
only showing top 20 rows

In [0]:
m.roc_auc_score
@pandas_udf("double")
def pandas_auc(label: pd.Series,probability: pd.Series) -> float:
    score = m.roc_auc_score(y_true= label, y_score=probability)
    return score


In [0]:
cls.training_predictions.select(pandas_auc('label','Average_score')).first()[0]

--------------------------------------------------------------------------- 
 IndexError Traceback (most recent call last)
 <command-1899558221395417> in <module> 
 ----> 1 cls . training_predictions . select ( pandas_auc ( 'label' , 'Average_score' ) ) [ 1 ] 

 /databricks/spark/python/pyspark/sql/dataframe.py in __getitem__ (self, item) 
 1403 return self . select ( * item ) 
 1404 elif isinstance ( item , int ) : 
 -> 1405 jc = self . _jdf . apply ( self . columns [ item ] ) 
 1406 return Column ( jc ) 
 1407 else : 

 IndexError : list index out of range

In [0]:
cls.training_predictions = cls.training_predictions.repartition(32)

In [0]:
cls.training_predictions.rdd.getNumPartitions()

Out[29]: 32

In [0]:
cls.training_predictions = cls.training_predictions.repartition(1)

In [0]:
pnd = cls.training_predictions.toPandas()

In [0]:
from sklearn import metrics as m
print(m.accuracy_score(pnd.label,pnd.Prediction))
print(m.roc_auc_score(pnd.label,pnd.Average_Score))
print(m.precision_score(pnd.label,pnd.Prediction))
print(m.recall_score(pnd.label,pnd.Prediction))

0.8827381880555123
0.9301852229249494
0.8277380297193175
0.7740607308286156

In [0]:
from sklearn import metrics as m
print(m.accuracy_score(pnd.label,pnd.Prediction))
print(m.roc_auc_score(pnd.label,pnd.Average_Score))
print(m.precision_score(pnd.label,pnd.Prediction,pos_label=0))
print(m.recall_score(pnd.label,pnd.Prediction,pos_label=0))

0.8827381880555123
0.9301852229249494
0.9044821583986075
0.9299776286353467

In [0]:
print(m.classification_report(pnd.label,pnd.Prediction))

precision recall f1-score support

 0.0 0.86 0.91 0.88 4516
 1.0 0.76 0.65 0.70 1937

 accuracy 0.83 6453
 macro avg 0.81 0.78 0.79 6453
weighted avg 0.83 0.83 0.83 6453

In [0]:
report = pd.DataFrame(m.classification_report(pnd.label,pnd.Prediction,output_dict=True))
# report['index'] = report.index
report.insert(0, 'index', report.index)
spark_report = spark.createDataFrame(report)
spark_report.show()

+---------+------------------+------------------+------------------+------------------+------------------+
 index| 0.0| 1.0| accuracy| macro avg| weighted avg|
+---------+------------------+------------------+------------------+------------------+------------------+
precision|0.8598659405111018|0.7552114353782013|0.8326359832635983|0.8075386879446516|0.8284517492136544|
 recall|0.9089902568644819|0.6546205472379969|0.8326359832635983|0.7818054020512394|0.8326359832635983|
 f1-score|0.8837459634015069|0.7013274336283186|0.8326359832635983|0.7925366985149127|0.8289893087957939|
 support| 4516.0| 1937.0|0.8326359832635983| 6453.0| 6453.0|
+---------+------------------+------------------+------------------+------------------+------------------+

In [0]:
from typing import Iterator
import pandas as pd
from sklearn import metrics as m

# df = spark.createDataFrame([(1, 21), (2, 30)], ("id", "age"))
# l = 3
# cols = df.columns[:-2]
def pandas_filter(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for pdf in iterator:
      report = pd.DataFrame(m.classification_report(pdf.label,pdf.Prediction,output_dict=True))
      report.insert(0, 'index', report.index)
      yield report

over_all_report =cls.training_predictions.mapInPandas(pandas_filter, schema=spark_report.schema)

In [0]:
over_all_report.show(50)

+---------+------------------+------------------+------------------+------------------+------------------+
 index| 0.0| 1.0| accuracy| macro avg| weighted avg|
+---------+------------------+------------------+------------------+------------------+------------------+
precision|0.8524822695035461| 0.721030042918455|0.8198294243070362|0.7867561562110006|0.8143639266131999|
 recall|0.9024024024024024|0.6176470588235294|0.8198294243070362|0.7600247306129659|0.8198294243070362|
 f1-score|0.8767323121808899|0.6653465346534654|0.8198294243070362|0.7710394234171776|0.8154349438573724|
 support| 666.0| 272.0|0.8198294243070362| 938.0| 938.0|
precision|0.8432601880877743|0.7072072072072072|0.8081395348837209|0.7752336976474907|0.8026024949641629|
 recall|0.8922056384742952|0.6108949416342413|0.8081395348837209|0.7515502900542683|0.8081395348837209|
 f1-score|0.8670427074939564|0.6555323590814196|0.8081395348837209|0.7612875332876881|0.8038355452357913|
 support| 603.0| 257.0|0.8081395348837209| 860.0| 860.0|
precision|0.8769497400346621| 0.8|0.8567774936061381| 0.838474870017331|0.8538254575434273|
 recall|0.9250457038391224|0.6978723404255319|0.8567774936061381|0.8114590221323272|0.8567774936061381|
 f1-score|0.9003558718861209|0.7454545454545456|0.8567774936061381|0.8229052086703332|0.8538062405415938|
 support| 547.0| 235.0|0.8567774936061381| 782.0| 782.0|
precision|0.8754789272030651|0.7526315789473684|0.8426966292134831|0.8140552530752168|0.8395909378249965|
 recall|0.9067460317460317| 0.6875|0.8426966292134831|0.7971230158730158|0.8426966292134831|
 f1-score|0.8908382066276802|0.7185929648241206|0.8426966292134831|0.8047155857259004|0.8405193719434942|
 support| 504.0| 208.0|0.8426966292134831| 712.0| 712.0|
precision| 0.841648590021692|0.7182320441988951|0.8068535825545171|0.7799403171102935|0.8026243551275677|
 recall|0.8838268792710706|0.6403940886699507|0.8068535825545171|0.7621104839705106|0.8068535825545171|
 f1-score|0.8622222222222221|0.6770833333333334|0.8068535825545171|0.7696527777777777|0.8036814209068882|
 support| 439.0| 203.0|0.8068535825545171| 642.0| 642.0|
precision|0.8578088578088578|0.7533333333333333|0.8307426597582038|0.8055710955710955|0.8264120680700991|
 recall| 0.908641975308642|0.6494252873563219|0.8307426597582038|0.7790336313324819|0.8307426597582038|
 f1-score|0.8824940047961632|0.6975308641975309|0.8307426597582038|0.7900124344968471|0.8269092267924292|
 support| 405.0| 174.0|0.8307426597582038| 579.0| 579.0|
precision|0.8493975903614458| 0.764|0.8260393873085339| 0.806698795180723|0.8222086420078563|
 recall|0.9052969502407705|0.6563573883161512|0.8260393873085339|0.7808271692784609|0.8260393873085339|
 f1-score|0.8764568764568764|0.7060998151571165|0.8260393873085339|0.7912783458069965|0.8222184685375875|
 support| 623.0| 291.0|0.8260393873085339| 914.0| 914.0|
precision|0.8617021276595744|0.7043795620437956|0.8196881091617934| 0.783040844851685|0.8161613849813226|
 recall|0.8888888888888888|0.6498316498316499|0.8196881091617934|0.7693602693602694|0.8196881091617934|
 f1-score|0.8750844024307899|0.6760070052539404|0.8196881091617934|0.7755457038423652| 0.817456734826965|
 support| 729.0| 297.0|0.8196881091617934| 1026.0| 1026.0|
+---------+------------------+------------------+------------------+------------------+------------------+

In [0]:
over_all_report.count()

Out[110]: 32

In [0]:
2400/4

Out[102]: 600.0

In [0]:
over_all_report.groupBy('index').avg('accuracy','macro avg',"weighted avg").show()

+---------+------------------+------------------+------------------+
 index| avg(accuracy)| avg(macro avg)| avg(weighted avg)|
+---------+------------------+------------------+------------------+
precision|0.8331775371772515|0.8100383089545686|0.8294883490442074|
 support|0.8331775371772515| 806.625| 806.625|
 recall|0.8331775371772515|0.7837870231254391|0.8331775371772515|
 f1-score|0.8331775371772515|0.7945445191042755|0.8295753763836694|
+---------+------------------+------------------+------------------+

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator as m
from pyspark.ml.evaluation import BinaryClassificationEvaluator as b
m_accuracy = m(predictionCol='Prediction',probabilityCol='Average_score',metricName='accuracy').evaluate(cls.training_predictions)
m_precision = m(predictionCol='Prediction',probabilityCol='Average_score',metricName='precisionByLabel').evaluate((cls.training_predictions))
recall=  m(predictionCol='Prediction',probabilityCol='Average_score',metricName='weightedRecall').evaluate((cls.training_predictions))


In [0]:
print(m_accuracy,m_precision,recall)
m_precision
recall

0.8225631489229815 0.8656778775851264 0.833410816674415
Out[47]: 0.833410816674415

In [0]:
np.mean([.1773,.1483,.17498])

Out[56]: 0.16686

In [0]:
rocessing took 0.11208266864999435 minutes to run ...
internal training took 0.7386802771000021 minutes to run ...
collecting models took 0.738199278733335 minutes to run ...
predicion on entire training data took 0.11993905096666518 minutes to run ...
predicion on entire validation data took 0.008740813349997248 minutes to run ...
spark model fitting took 0.4000970027833318 minutes to run ...
spark model evaluation took 0.0027818267500000124 minutes to run ...
spark results took 0.3215372752333299 minutes to run ...

In [0]:
processing took 2.262005022866665 minutes to run ...
internal training took 1.2922705989666647 minutes to run ...
collecting models took 1.1971958543833352 minutes to run ...
predicion on entire training data took 0.24874224846666568 minutes to run ...
predicion on entire validation data took 0.012835287483335378 minutes to run ...
spark model fitting took 4.871551131566665 minutes to run ...
spark model evaluation took 0.0035348839333323666 minutes to run ...
spark results took 0.36445042548333123 minutes to run ...

Out[7]: 10.178279999999999

In [0]:
df_val_spark.show(10)

+-------------------+-------------------+-------------------+-------------------+-------------------+-----+
 0| 1| 2| 3| 4|stabf|
+-------------------+-------------------+-------------------+-------------------+-------------------+-----+
-1.5901628929877183| 1.4794894085891301|-1.8167196043352458| 0.4562440829622908| 0.5067888737304939| 0|
-0.6823598910249344|-2.3104642580122055| 1.2720968883862374|-0.8871665073514782|-2.4151659980850706| 0|
-0.7707665047757042| -2.718455775405737| 1.0540298333031664| -1.487111213780111| -2.784663194514941| 0|
-1.4370601040537072| 1.0354791626657673|-0.2062861765302627| 1.4907946058092212|0.08014998870824122| 0|
-1.1369733054694549| 3.270673353703632|-2.5040978180653246| 1.4662211184074063| 2.3105395749708744| 0|
 -2.4302610074518| -1.805213462013654| 1.2281839907131742| 0.3397863199545863|-2.9636777161701024| 0|
-2.4194208383094375| 3.0556186529880263|-1.3764447757241407| 2.880766001275559| 1.3180775614681972| 0|
 2.164964653265702|-0.8118498688089199| 0.5748983448068492|-1.2087191344823678| 0.4785541977891434| 0|
 1.4038029421103435| 2.2973983851855087|-1.0753539913006644| 0.7581282564281021| 2.7884672141986835| 1|
 2.0586802157873216| 2.111427471240742|-1.1568040782443738|0.20688750597512484| 3.005397833191534| 1|
+-------------------+-------------------+-------------------+-------------------+-------------------+-----+
only showing top 10 rows

In [0]:
m_accuracy = m(predictionCol='prediction',probabilityCol='probability',metricName='accuracy').evaluate(fit_model.summary.predictions)
m_precision = m(predictionCol='prediction',probabilityCol='probability',metricName='precisionByLabel').evaluate(fit_model.summary.predictions)
recall=  m(predictionCol='prediction',probabilityCol='probability',metricName='weightedRecall').evaluate(fit_model.summary.predictions)

In [0]:
cls.model_name

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2911799196539681> in <module> 
 ----> 1 cls . model_name

 AttributeError : 'Binary_Classifier' object has no attribute 'model_name'

In [0]:
|model_id|      model_returend|  Accuracy|      AUC|
+--------+--------------------+----------+---------+
|       0|[80 03 63 73 6B 6...| 0.8076923|0.8469733|
|       1|[80 03 63 73 6B 6...| 0.8012048|0.8474444|
|       2|[80 03 63 73 6B 6...|0.82175225|0.8442235|
+--------+--------------------+----------+---------+


Out[25]: {'random_state': 32}

In [0]:
def exp(a=1, **kwargs):
  print(a,kwargs)

In [0]:
exp(3, **cls.custom_parameters)

3 {'random_state': 32}

In [0]:
dict(zip (list(range(5)) , [i*1.0 for i in range(5)]))

Out[59]: {0: 0.0, 1: 1.0, 2: 2.0, 3: 3.0, 4: 4.0}

In [0]:
df = spark.createDataFrame(pd.DataFrame(dict(a=[0,.3],b=[1,.9],c=[1,.3],d=[1,.5],label=[1,0],mean=[0.0,0.0])))

In [0]:
df.show()

+---+---+---+---+-----+----+
 a| b| c| d|label|mean|
+---+---+---+---+-----+----+
0.0|1.0|1.0|1.0| 1| 0.0|
0.3|0.9|0.3|0.5| 0| 0.0|
+---+---+---+---+-----+----+

In [0]:
df.withColumn('label',expr("label - 1.0")).show()

+---+---+---+---+-----+----+
 a| b| c| d|label|mean|
+---+---+---+---+-----+----+
0.0|1.0|1.0|1.0| 0.0| 0.0|
0.3|0.9|0.3|0.5| -1.0| 0.0|
+---+---+---+---+-----+----+

In [0]:
Window.partitionBy()

+--------------------+---+
 _1| _2|
+--------------------+---+
[0.0, 1.0, 1.0, 1...| 0|
[0.3, 0.9, 0.3, 0...| 1|
+--------------------+---+

In [0]:
from pyspark.ml.classification import LogisticRegression as lr
from pyspark.ml.feature import VectorAssembler
vec = VectorAssembler(inputCols=['a','b','c','d'],outputCol='features')
vec = vec.transform(df)
vec.show()
model = lr()
fit_model = model.fit(vec)

+---+---+---+---+-----+----+-----------------+
 a| b| c| d|label|mean| features|
+---+---+---+---+-----+----+-----------------+
0.0|1.0|1.0|1.0| 1| 0.0|[0.0,1.0,1.0,1.0]|
0.3|0.9|0.3|0.5| 0| 0.0|[0.3,0.9,0.3,0.5]|
+---+---+---+---+-----+----+-----------------+

In [0]:
fit_model.summary.predictions.show(truncate=False)

+---+---+---+---+-----+----+-----------------+--------------------------------------+------------------------------------------+----------+
a |b |c |d |label|mean|features |rawPrediction |probability |prediction|
+---+---+---+---+-----+----+-----------------+--------------------------------------+------------------------------------------+----------+
0.0|1.0|1.0|1.0|1.0 |0.0 |[0.0,1.0,1.0,1.0]|[-19.90811163980995,19.90811163980995]|[2.2595240525230098E-9,0.9999999977404759]|1.0 |
0.3|0.9|0.3|0.5|0.0 |0.0 |[0.3,0.9,0.3,0.5]|[20.30639060726522,-20.30639060726522]|[0.9999999984827868,1.5172131956485374E-9]|0.0 |
+---+---+---+---+-----+----+-----------------+--------------------------------------+------------------------------------------+----------+

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator as m
from pyspark.ml.evaluation import BinaryClassificationEvaluator as b
m_accuracy = m(predictionCol='prediction',probabilityCol='probability',metricName='accuracy').evaluate(fit_model.summary.predictions)
m_precision = m(predictionCol='prediction',probabilityCol='probability',metricName='precisionByLabel').evaluate(fit_model.summary.predictions)
recall=  m(predictionCol='prediction',probabilityCol='probability',metricName='weightedRecall').evaluate(fit_model.summary.predictions)
b_accuracy = b(rawPredictionCol='rawPrediction').evaluate(fit_model.summary.predictions)

In [0]:
m_precision

Out[42]: 1.0

In [0]:
recall

Out[40]: 1.0

In [0]:
from typing import Iterator
import pandas as pd

# df = spark.createDataFrame([(1, 21), (2, 30)], ("id", "age"))
l = 3
cols = df.columns[:-2]
def pandas_filter(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
    for pdf in iterator:
      pdf['mean'] = pdf[cols].mean(axis=1)
      yield pdf

df0.mapInPandas(pandas_filter, schema=df.schema).show()

+---+---+---+---+-----+----+
 a| b| c| d|label|mean|
+---+---+---+---+-----+----+
0.0|1.0|1.0|1.0| 1|0.75|
0.3|0.9|0.3|0.5| 0| 0.5|
+---+---+---+---+-----+----+

In [0]:
df1.show()

+---+---+---+---+-----+----+
 a| b| c| d|label|mean|
+---+---+---+---+-----+----+
0.0|1.0|1.0|1.0| 1|0.75|
0.3|0.9|0.3|0.5| 0| 0.5|
+---+---+---+---+-----+----+

In [0]:
df1.select("*",when(df1.mean==.5,1).otherwise(0).alias("Prediction")).show()

+---+---+---+---+-----+----+----------+
 a| b| c| d|label|mean|Prediction|
+---+---+---+---+-----+----+----------+
0.0|1.0|1.0|1.0| 1|0.75| 0|
0.3|0.9|0.3|0.5| 0| 0.5| 1|
+---+---+---+---+-----+----+----------+

In [0]:
df1.schema

Out[22]: StructType(List(StructField(a,DoubleType,true),StructField(b,DoubleType,true),StructField(c,DoubleType,true),StructField(d,DoubleType,true),StructField(label,LongType,true),StructField(mean,DoubleType,true)))

In [0]:
df0 

In [0]:
spark.createDataFrame(df2).show()

+---+---+---+---+-----+----+
 a| b| c| d|label|mean|
+---+---+---+---+-----+----+
0.0|1.0|1.0|1.0| 1|0.75|
+---+---+---+---+-----+----+

In [0]:
from pyspark.ml import evaluation

In [0]:
evaluation.MulticlassClassificationEvaluator.metricName

In [0]:
def internal_training(key,pdf):
  # get the sample id
  id = pdf[group_column].iloc[0]
  # create a smaller dataset
  small = pdf.head(100)
  #call setup
  cls1= setup(small.drop([group_column],axis=1), target = target, session_id=sessionid,silent=True,html=False,preprocess=False,use_gpu=gpu)
  # Now change the it with original (since we do not need preprocessing)
  set_config('X_train',pdf.drop([group_column,target],axis=1))
  set_config('y_train',pdf[target])

  #Modeling
  # if create model is selected
  if model_name == "Auto Select":
    model = compare_models(n_select=1,cross_validation=False) 
  else:
    model = create_model(model_name ,cross_validation=False,**cp_create_model) # while unpacking the custom parameters



  # try to get AUC
  try:
    prediction_prob = model.predict_proba (pdf.drop([group_column,target],axis=1))
    AUC = m.roc_auc_score(pdf[target],prediction_prob[:, 1])
  except:
    AUC = 0.0


  # get prediction & accuracy
  prediction = model.predict(pdf.drop([group_column,target],axis=1))  
  accuracy = m.accuracy_score(y_true=pdf[target],y_pred=prediction)
  # convert to df
  return_df = pd.DataFrame(dict(model_id = id,model_returend = [pickle.dumps(model)], Accuracy = accuracy, AUC = AUC))


  return(return_df)


# Call the training function with sampled data
  #=============================================
  return_internal_training  = self.df_sample.groupby(self.group_column).applyInPandas(internal_training, schema=self.internal_training_schema)

In [0]:
df_train_spark = df_train_spark.repartition(32)

In [0]:
#schema
schema_exp = spark.createDataFrame(pd.DataFrame(dict(model_id=[1.0],model_name=['name'],auc=[0.0]))).schema
schema_exp

Out[4]: StructType(List(StructField(model_id,DoubleType,true),StructField(model_name,StringType,true),StructField(auc,DoubleType,true)))

In [0]:
df_train_spark.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
 0| 1| 2| 3| 4|stabf|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
 0.4841157802636492| 1.132850812153862| -0.5707566310628132|-0.07908223376779393|-0.05779279948368173| 1|
 3.0165388490378118| -0.6888737345570872| 0.9542711630479714| -1.9573780405663248| 2.8610325707612856| 0|
 -2.666942183973619| -1.1906254127520637| 0.6169464252445396| 1.3312990506761342| -1.9385163025604524| 0|
 -1.7167385963165098|-0.08381114986113691| -2.050451700483732| 1.279114924265886| -0.750654905755199| 1|
 -1.2193358582715519| 0.30838939076109007| -1.733618372438305| 0.9870738062865286| -0.6621043565699303| 1|
 -0.9904945257347679| 1.233388425721143| -0.8822115473010734| 0.8306092097068333| -1.364849158407123| 1|
 -1.7922463450585717| -1.544843678527| -1.1732255780352299| 1.0428737633030836| -0.2230858181559789| 0|
-0.44251739233959875| -0.6902696000354217|-0.11896476889673091| 0.2004098992461426| 0.07678249197500482| 0|
 -1.5006944936175128| 0.12898365437150927| -1.8815507919665724| 1.1522865060860281| -0.7511713764474458| 1|
 -0.8084380716847155| -1.4584283341211695| 0.12639557054497064| 0.29610551319779654| 0.13516425685332012| 0|
 -0.9524267012091576| 2.0079324899606332|-0.10039972482059756| 0.7787274139860443| -2.1213163792463505| 1|
 -1.3328730529817334| -0.4231152326315738| 0.3157070185867732| 0.682488095378585| -1.0812432028445542| 0|
 -0.5848153801869187| -1.6694761666128786| -3.409784461159406| 0.649739958857177| 1.8232569428923187| 0|
 2.332189103040692| 0.9321971395284531| -2.496428433558494| -0.8959535369164667| 2.510135199380972| 1|
 3.260325043864053|-0.27950333854682896| 1.7604817190865334| -2.1705975443727588| 2.5180347062544852| 0|
 -1.5047462892339292| -1.3012154016471946| -1.2685258307014518| 0.915671112113852|-0.07663603612538675| 0|
 -1.5781051094415774| 0.7790584398901276| -1.289812765348312| 1.1813282619332448| -1.4625082116132377| 1|
 0.2462032566327942| -1.5098506777473868| -1.6028451721488102|-0.07327079371273702| 1.8004968190295063| 0|
 0.03941581419200535| -0.4658780995669195| -2.089568033057555| 0.22660305494138608| 1.1293990753209633| 0|
 -1.2155894475419822|0.030025927802749486| -1.9629949816286916| 0.9882435709576579| -0.3938013041649294| 1|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 20 rows

In [0]:
from typing import Iterator
import pandas as pd

def pandas_filter(iterator: Iterator[pd.DataFrame]) -> Iterator[pd.DataFrame]:
  id = 0
  for pdf in iterator:
    #df = pd.DataFrame()
    cls1= setup(pdf, target = 'stabf',silent=True,html=False,preprocess=False)
    #model = create_model('lr' ,cross_validation=False)
    model = compare_models(n_select=1,cross_validation=False)
    
    pdf = pdf.head(1)
    pdf['model_id'] = id
    pdf['model_name'] = str(model.__class__)
    pdf['auc'] = 0.90
    pdf = pdf[['model_id','model_name','auc']]
    id =+1
    
    yield pdf


In [0]:
df_train_spark.mapInPandas(pandas_filter, schema=schema_exp).show(truncate=False)

+--------+------------------------------------------------------------------+---+
model_id|model_name |auc|
+--------+------------------------------------------------------------------+---+
0.0 |<class 'sklearn.linear_model._logistic.LogisticRegression'> |0.9|
0.0 |<class 'catboost.core.CatBoostClassifier'> |0.9|
0.0 |<class 'sklearn.discriminant_analysis.LinearDiscriminantAnalysis'>|0.9|
0.0 |<class 'sklearn.neighbors._classification.KNeighborsClassifier'> |0.9|
0.0 |<class 'sklearn.linear_model._logistic.LogisticRegression'> |0.9|
0.0 |<class 'sklearn.linear_model._ridge.RidgeClassifier'> |0.9|
0.0 |<class 'sklearn.ensemble._gb.GradientBoostingClassifier'> |0.9|
0.0 |<class 'sklearn.ensemble._gb.GradientBoostingClassifier'> |0.9|
0.0 |<class 'sklearn.linear_model._logistic.LogisticRegression'> |0.9|
0.0 |<class 'sklearn.neighbors._classification.KNeighborsClassifier'> |0.9|
0.0 |<class 'sklearn.neighbors._classification.KNeighborsClassifier'> |0.9|
0.0 |<class 'sklearn.neighbors._classification.KNeighborsClassifier'> |0.9|
0.0 |<class 'sklearn.linear_model._logistic.LogisticRegression'> |0.9|
0.0 |<class 'sklearn.discriminant_analysis.LinearDiscriminantAnalysis'>|0.9|
0.0 |<class 'sklearn.linear_model._logistic.LogisticRegression'> |0.9|
0.0 |<class 'sklearn.linear_model._logistic.LogisticRegression'> |0.9|
0.0 |<class 'xgboost.sklearn.XGBClassifier'> |0.9|
0.0 |<class 'sklearn.linear_model._ridge.RidgeClassifier'> |0.9|
0.0 |<class 'sklearn.linear_model._ridge.RidgeClassifier'> |0.9|
0.0 |<class 'sklearn.linear_model._logistic.LogisticRegression'> |0.9|
+--------+------------------------------------------------------------------+---+
only showing top 20 rows

In [0]:
from pycaret.classification import *

In [0]:
# pnd = df_val_spark.toPandas
s = setup(df_val_spark.toPandas(),'stabf',silent=True,html=False,preprocess=False)

Setup Succesfully Completed!
 Description Value
0 session_id 3687
1 Target stabf
2 Target Type Binary
3 Label Encoded 0: 0, 1: 1
4 Original Data (2023, 6)
5 Missing Values False
6 Numeric Features 5
7 Categorical Features 0
8 Transformed Train Set (1416, 5)
9 Transformed Test Set (607, 5)
10 Shuffle Train-Test True
11 Stratify Train-Test False
12 Fold Generator StratifiedKFold
13 Fold Number 10
14 CPU Jobs -1
15 Use GPU False
16 Log Experiment False
17 Experiment Name clf-default-name
18 USI a07b
19 Fix Imbalance False
20 Fix Imbalance Method SMOTE

In [0]:
mod = create_model('lr')

Accuracy AUC Recall Prec. F1 Kappa MCC
0 0.9648 0.9890 0.9250 0.9487 0.9367 0.9123 0.9125
1 0.9577 0.9946 0.9250 0.9250 0.9250 0.8956 0.8956
2 0.9437 0.9699 0.9000 0.9000 0.9000 0.8608 0.8608
3 0.9648 0.9968 0.9250 0.9487 0.9367 0.9123 0.9125
4 0.9437 0.9863 0.9000 0.9000 0.9000 0.8608 0.8608
5 0.9718 0.9799 0.9250 0.9737 0.9487 0.9293 0.9299
6 0.9504 0.9861 0.9250 0.9024 0.9136 0.8788 0.8789
7 0.9716 0.9918 0.9500 0.9500 0.9500 0.9302 0.9302
8 0.9574 0.9931 0.9250 0.9250 0.9250 0.8953 0.8953
9 0.9716 0.9958 0.9250 0.9737 0.9487 0.9291 0.9297
Mean 0.9598 0.9883 0.9225 0.9347 0.9284 0.9005 0.9006
SD 0.0105 0.0079 0.0135 0.0270 0.0181 0.0255 0.0256

In [0]:
mod.__class__

Out[27]: sklearn.linear_model._logistic.LogisticRegression

In [0]:
class example():
  def __init__(self,a,b):
    self.a = a
    self.b = b
    

In [0]:
cls1 = example(10,20)

In [0]:
filename = '/dbfs/FileStore/Fahad/SDM_Cash_Rec/cls_object.sav'
pickle.dump(cls1, open(filename, 'wb'))

In [0]:
file_path= '/dbfs/FileStore/Fahad/SDM_Cash_Rec/cls_object.sav'
loaded_cls1  = pickle.load(open(file_path,'rb'))

In [0]:
loaded_cls1.a

Out[16]: 10

In [0]:
loaded_cls1.b

Out[17]: 20

In [0]:
(.015*.15)/.15

Out[24]: 0.015

In [0]:
0.015*10000

Out[25]: 150.0

In [0]:
10000-150

Out[26]: 9850

In [0]:
1.5*15

Out[27]: 22.5

File "<command-1899558221421477>" , line 1 
 2% 
 ^
 SyntaxError : invalid syntax